<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/3%20Aprendizaje%20profundo%20(II)/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision import transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset

from sklearn.model_selection import train_test_split

We are going to create a GAN to generate files that are similar to Bob Ross

In [2]:
# Access to the BOB ROSS images folder
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#!ls gdrive/MyDrive/UNIVERSIDAD/Master\ Propio\ IA\ UV/3\ Aprendizaje\ profundo\ (II)/Datasets/Bob\ Ross/train/images
directory = 'gdrive/MyDrive/UNIVERSIDAD/Master Propio IA UV/3 Aprendizaje profundo (II)/Datasets/Bob Ross/train/images'

In [4]:
transform = transforms.Compose([
  transforms.ToTensor()
])

In [5]:
directory = "gdrive/MyDrive/UNIVERSIDAD/Master Propio IA UV/3 Aprendizaje profundo (II)/Datasets/Bob Ross/train"

In [6]:
folder = datasets.ImageFolder(root = directory, transform=transform)

In [7]:
folder

Dataset ImageFolder
    Number of datapoints: 500
    Root location: gdrive/MyDrive/UNIVERSIDAD/Master Propio IA UV/3 Aprendizaje profundo (II)/Datasets/Bob Ross/train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [8]:
batch_size = 20

folder_loader = DataLoader(folder, batch_size=batch_size, shuffle=True, num_workers=2)

In [9]:
def train_val_dataset(dataset, val_split=0.25):
  train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split, shuffle=True)
  datasets = {}
  datasets['train'] = Subset(dataset, train_idx)
  datasets['val'] = Subset(dataset, val_idx)
  return datasets

In [10]:
datasets = train_val_dataset(folder)

### Small experiment to check how grad works in PyTorch tensors

In [26]:
x = torch.randn(5)
w = torch.randn(3,5).requires_grad_()
b = torch.randn(3).requires_grad_()

y = torch.randn(3)

In [22]:
def linear(x, w, b):
  return x @ w.t() + b

def mse(pred, real):
  return (pred - real).pow(2).mean()

In [28]:
result = linear(x, w, b)
loss = mse(result, y)

In [29]:
loss

tensor(5.4519, grad_fn=<MeanBackward0>)

In [30]:
loss.backward()

In [31]:
w.grad

tensor([[ 0.9431, -0.0135, -0.7908,  0.0409,  0.9231],
        [ 1.6658, -0.0239, -1.3967,  0.0723,  1.6304],
        [ 3.4015, -0.0488, -2.8520,  0.1475,  3.3292]])

In [32]:
w.data

tensor([[ 1.2697, -0.6939,  0.1321, -0.1466, -0.3914],
        [-1.2386, -1.8907, -1.8679, -1.9964,  0.7666],
        [ 0.6081,  0.5699,  0.1135, -0.5932,  0.5787]])

In [33]:
w

tensor([[ 1.2697, -0.6939,  0.1321, -0.1466, -0.3914],
        [-1.2386, -1.8907, -1.8679, -1.9964,  0.7666],
        [ 0.6081,  0.5699,  0.1135, -0.5932,  0.5787]], requires_grad=True)

In [34]:
b.data

tensor([ 0.5760,  0.7353, -1.4715])

In [35]:
b.grad

tensor([-0.6515, -1.1507, -2.3496])

In [46]:
a1 = torch.randn(1,3).requires_grad_()
a1.shape

torch.Size([1, 3])

In [43]:
aa1 = a1.squeeze()
aa1.shape

torch.Size([3])

In [45]:
a2 = torch.randn(2,3).requires_grad_()
a2.shape

torch.Size([2, 3])

In [47]:
aa2 = a2.squeeze()
aa2.shape #The squeeze function does nothing because there is no dimension to squeeze

torch.Size([2, 3])

### Continuing with building the Bob Ross generator